# PASSNYC Exploration | Data for Good

**This code is based on previous works and it's still under construction**

## Table of contents

* Introduction
* Data pre-processing
* Data visualization
* Conclusion

## Introduction

Before we start, we will to try to understand our data. We have two datasets: "2016 School Explorer" (from now on **df**) and "D5 SHSAT Registrations and Testers" (from now on **registrations**). 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('../input/2016 School Explorer.csv')
registrations = pd.read_csv('../input/D5 SHSAT Registrations and Testers.csv')

df.head()

In [ ]:
print('----- Dataframe Information -----')
print('---------------------------------')
print(df.info())
print('----- Registrations Information -----')
print('-------------------------------------')
print(registrations.info())

The dataframe *df* has a total of 161 columns of different types, most of them integers. Let's take a deeper look into each column. On the other hand, the *registrations* dataframe has a total of 7 columns. Let's get into more detail, starting with *df*.

In [ ]:
for i in df.columns:
    print(i,',' ,'{:.1%}'.format(np.mean(df[i].isnull())),'nulls',',',type(df[i][0]), 
    df[i].nunique(), 'unique values')

We see that there are a few columns with a crazy amount of null elements, so we will get rid of those columns ("Adjusted grade", "New?", "Other Location Code in LCGMS"). At some point we will also deal with the column "School Income Estimate", which has a 30% of null elements but gives a relevant information.

How can we fill the missing values in the column? The most common thing to do is taking the mean or median, depending on the type of data, within a certain range. We will make a correlation map between this column and others, to see where we can take the mean.

In [ ]:
df['School Income Estimate'] = df['School Income Estimate'].str.replace('$', '')
df['School Income Estimate'] = df['School Income Estimate'].str.replace(',', '')
df['School Income Estimate'] = df['School Income Estimate'].str.replace('.', '')
df['School Income Estimate'] = df['School Income Estimate'].astype(float)

df = df.drop(columns=['Adjusted Grade', 'New?', 'Other Location Code in LCGMS'])
corrcolumns = ['SED Code', 'District', 'Latitude', 'Longitude', 'Zip', 'Economic Need Index'] # Community School? would be interesting aswell
for i in corrcolumns:
    print('The correlation value between School Income Estimate and ', i, 'is: ', df['School Income Estimate'].corr(df[i]))


Taking only the previous examples, we can see that **school income estimate** and **economic need index** are almost completely anti-correlated. That makes sense; if you have an high income, you will have a small economic need (index), and viceversa. Thus, this can be a good aspect to analyse, and we will use it when it comes to data pre-processing.

# Registrations

Before we go into detail of the dataframe, let's do a recap. So what exactly is the SHSAT exam? There are **nine** specialized High Schools in New York, which offer a more rigorous curriculum than most other public high schools in the city. The schools are intended to serve the needs of students who excel academically and artistically, according to the Department of Education. The schools are:

* **Bronx High School of Science**

* **The Brooklyn Latin School**

* **Brooklyn Technical High School**

* **High School of American Studies at Lehman College**, in the Bronx

* **High School for Math, Science and Engineering at City College**, in Manhattan

* **Queens High School for the Sciences at York College**

* **Staten Island Technical High School**

* **Stuyvesant High School**, in Manhattan

The ninth school, Fiorello H. LaGuardia High School of Music and Art and Performing Arts, on the Upper West Side,  uses student auditions and academic records, not  SHSAT scores, to determine admissions. So, in terms of our problem, we have **eight** specialized high schools.

Each of the eight schools has a cutoff score, which is the minimum score that students must get to be offered a seat.

## The problem

Numbers don't lie. The number of black and latino students admitted to specialized high schools is **extremely** low. Only 10 percent of New York City’s public school students who are black or Latino received offers to attend a specialized high school last year, even though 67 percent of New York public school students are black or Latino. Asians make up 62 percent of students at specialized high schools and white students make up 24 percent, though only 16 percent of public school students are Asian and 15 percent are white.

The SHSAT helps determine who is most likely to enjoy success. The best way for the city to increase diversity at these schools is to improve the students’ primary school education and to provide greater access to quality, affordable test prep in advance of the SHSAT. This is the problem we have to solve.

As stated in the dataframe description, the **registrations** dataframe contains only data from district 5 (Central Harlem, Manhattan). Let's explore a bit more the dataframe.

In [ ]:
registrations.head()

In [ ]:
for i in registrations.columns:
    print(i,',' ,'{:.1%}'.format(np.mean(registrations[i].isnull())),'nulls',',',type(registrations[i][0]), 
    registrations[i].nunique(), 'unique values')

In [ ]:
print('Total number of students who registered for the SHSAT:',np.sum(registrations['Number of students who registered for the SHSAT']))
print('Total number of students who took the SHSAT:', np.sum(registrations['Number of students who took the SHSAT']))

As opposed to the **df** dataframe, this one is clean! No null rows, so another thing we do not need to deal with. 

There are two columns worth noticing: 'Number of students who took the SHSAT' and 'Number of students who registered for the SHSAT'. We will create a new column for the dataframe which is the percentage of students who registered for the SHSAT **and** took the exam.

In [ ]:
registrations['% of taken over registered students'] = (registrations['Number of students who took the SHSAT'])*100/(registrations['Number of students who registered for the SHSAT'])

registrations.head()

# *District codes in New York City*

In [ ]:
df['District'].unique()



There are 32 district codes in the New York City:

* District 1 to 6: Manhattan Borough
* District 7 to 12: The Bronx Borough
* District 13 to 23 and district 32: Brooklyn Borough
* District 24 to 30: Queens Borough
* District 31: Staten Island Borough

![](http://)For those of us who are not familiar with the area, this helps quite a lot when it comes to understanding the data. 

The district map looks like this:

![](https://imageshack.com/a/img922/4818/Nl4VZg.jpg)

For convenience, I've created a column that specifies each of the different Boroughs, which makes it easier to understand.

In [ ]:
manh_districts = [1,2,3,4,5,6]
bronx_districts = [7, 8,9,10,11,12]
brook_districts = [13,14,15,16,17,18,19,20,21,22,23,32]
queens_districts = [24,25,26,27,28,29,30]
staten_districts = [31]


df.loc[df['District'].isin(manh_districts), 'Borough'] = 'Manhattan'
df.loc[df['District'].isin(bronx_districts), 'Borough'] = 'Bronx'
df.loc[df['District'].isin(brook_districts), 'Borough'] = 'Brooklyn'
df.loc[df['District'].isin(queens_districts), 'Borough'] = 'Queens'
df.loc[df['District'].isin(staten_districts), 'Borough'] = 'Staten Island'

In [ ]:
sns.pairplot(df[['Borough','Latitude','Longitude','Economic Need Index']], kind="scatter", hue="Borough", plot_kws=dict(s=80, edgecolor="white", linewidth=2.5))


In [ ]:
df.loc[:,'Student Attendance Rate'] = df.loc[:,'Student Attendance Rate'] .str.replace('%', '')
df.loc[:,'Student Attendance Rate'] = df.loc[:,'Student Attendance Rate'] .astype(float)
plt.figure(figsize=(15,10))
sns.boxplot(x="Borough", y="Student Attendance Rate", data=df, dodge=False);
plt.show()

In [ ]:
df.loc[:,'Percent Black'] = df.loc[:,'Percent Black'].str.replace('%', '')
df.loc[:,'Percent Black'] = df.loc[:,'Percent Black'].astype(float)
df.loc[:,'Percent Hispanic'] = df.loc[:,'Percent Hispanic'].str.replace('%', '')
df.loc[:,'Percent Hispanic'] = df.loc[:,'Percent Hispanic'].astype(float)
df.loc[:,'Percent Asian']  = df.loc[:,'Percent Asian'].str.replace('%', '')
df.loc[:,'Percent Asian']  = df.loc[:,'Percent Asian'].astype(float)
df.loc[:,'Percent White'] = df.loc[:,'Percent White'] .str.replace('%', '')
df.loc[:,'Percent White'] = df.loc[:,'Percent White'] .astype(float)


sns.lmplot('Economic Need Index', 'Percent Black', data=df, hue='Borough', fit_reg=True, size = 15)
plt.title('Percent of Black students in the different Boroughs of NYC')

sns.lmplot('Economic Need Index', 'Percent Hispanic', data=df, hue='Borough', fit_reg=True,  size = 15)
plt.title('Percent of Hispanic students in the different Boroughs of NYC')

sns.lmplot('Economic Need Index', 'Percent Asian', data=df, hue='Borough', fit_reg=True,  size = 15)
plt.title('Percent of Asian students in the different Boroughs of NYC')

sns.lmplot('Economic Need Index', 'Percent White', data=df, hue='Borough', fit_reg=True,  size = 15)
plt.title('Percent of WHite students in the different Boroughs of NYC')


To make sure we have relevant information from all 32 districts, we can plot the amount of inputs for each district.

In [ ]:
plt.figure(figsize=(10,10))
ax = sns.countplot(df['District'],label="Count", order = df['District'].value_counts().index)


We will now proceed to explore our dataset for each borough. To do so, we will divide our dataframe into 5 different dataframes, and calculate the median income for each of them (keeping in mind that we have not filled the missing values yet). The median is more relevant than the mean, since its the answer of the following question: what does the person earn who earns exactly more than 50% and less than 50% of the population? 

In [ ]:
df_manha = df[(df['District'].isin(manh_districts))]
df_bronx = df[(df['District'].isin(bronx_districts))]
df_brook = df[(df['District'].isin(brook_districts))]
df_queens = df[(df['District'].isin(queens_districts))]
df_staten = df[(df['District'].isin(staten_districts))]

median = df['School Income Estimate'].median()
median_manha = df_manha['School Income Estimate'].median()
median_bronx = df_bronx['School Income Estimate'].median()
median_brook = df_brook['School Income Estimate'].median()
median_queens = df_queens['School Income Estimate'].median()
median_staten = df_staten['School Income Estimate'].median()
print('Global median:', median)
print('Median school income in Manhattan: $', median_manha)
print('Median school income in The Bronx: $', median_bronx)
print('Median school income in Brooklyn: $', median_brook)
print('Median school income in Queens: $', median_queens)
print('Median school income in Staten Island: $', median_staten)

## Manhattan Borough

As we said, district codes 1, 2, 3, 4, 5 and 6 correspond to the Manhattan Borough. We will filter our dataframe with those districts and analyse the data.

In [ ]:
import folium
from folium import plugins
from io import StringIO
import folium 

df_manha['Economic Need Index'] = df_manha['Economic Need Index'].fillna((df_manha['Economic Need Index'].mean()))


#colors = ['red', 'yellow', 'dusty purple', 'blue', 'white', 'brown', 'green', 'purple', 'orange', 'grey', 'coral']
colors = ['chartreuse', 'limegreen', 'yellowgreen', 'y', 'olive', 'indianred', 'firebrick', 'tomamto', 'orangered', 'red']
d = (df_manha['Economic Need Index']*10).astype('int')
cols = [colors[int(i)] for i in d]


map_osm2 = folium.Map([df_manha['Latitude'][0], df_manha['Longitude'][0]], zoom_start=10.2,tiles='cartodbdark_matter')

for lat, long, col in zip(df_manha['Latitude'], df_manha['Longitude'], cols):
    #rown = list(rown)
    folium.CircleMarker([lat, long], color=col, fill=True, radius=2).add_to(map_osm2)

map_osm2

In [ ]:
df_bronx['Economic Need Index'] = df_bronx['Economic Need Index'].fillna((df_bronx['Economic Need Index'].mean()))


#colors = ['red', 'yellow', 'dusty purple', 'blue', 'white', 'brown', 'green', 'purple', 'orange', 'grey', 'coral']
colors = ['chartreuse', 'limegreen', 'yellowgreen', 'y', 'olive', 'indianred', 'firebrick', 'tomamto', 'orangered', 'red']
d = (df_bronx['Economic Need Index']*10).astype('int')
cols = [colors[int(i)] for i in d]


map_osm2 = folium.Map([df_bronx['Latitude'].iloc[0], df_bronx['Longitude'].iloc[0]], zoom_start=10.2,tiles='cartodbdark_matter')

for lat, long, col in zip(df_bronx['Latitude'], df_bronx['Longitude'], cols):
    #rown = list(rown)
    folium.CircleMarker([lat, long], color=col, fill=True, radius=2).add_to(map_osm2)

map_osm2

In [ ]:
df_brook['Economic Need Index'] = df_brook['Economic Need Index'].fillna((df_brook['Economic Need Index'].mean()))


#colors = ['red', 'yellow', 'dusty purple', 'blue', 'white', 'brown', 'green', 'purple', 'orange', 'grey', 'coral']
colors = ['chartreuse', 'limegreen', 'yellowgreen', 'y', 'olive', 'indianred', 'firebrick', 'tomamto', 'orangered', 'red']
d = (df_brook['Economic Need Index']*10).astype('int')
cols = [colors[int(i)] for i in d]


map_osm2 = folium.Map([df_brook['Latitude'].iloc[0], df_brook['Longitude'].iloc[0]], zoom_start=10.2,tiles='cartodbdark_matter')

for lat, long, col in zip(df_brook['Latitude'], df_brook['Longitude'], cols):
    #rown = list(rown)
    folium.CircleMarker([lat, long], color=col, fill=True, radius=2).add_to(map_osm2)

map_osm2

In [ ]:
df_queens['Economic Need Index'] = df_queens['Economic Need Index'].fillna((df_queens['Economic Need Index'].mean()))


#colors = ['red', 'yellow', 'dusty purple', 'blue', 'white', 'brown', 'green', 'purple', 'orange', 'grey', 'coral']
colors = ['chartreuse', 'limegreen', 'yellowgreen', 'y', 'olive', 'indianred', 'firebrick', 'tomamto', 'orangered', 'red']
d = (df_queens['Economic Need Index']*10).astype('int')
cols = [colors[int(i)] for i in d]


map_osm2 = folium.Map([df_queens['Latitude'].iloc[0], df_queens['Longitude'].iloc[0]], zoom_start=10.2,tiles='cartodbdark_matter')

for lat, long, col in zip(df_queens['Latitude'], df_queens['Longitude'], cols):
    #rown = list(rown)
    folium.CircleMarker([lat, long], color=col, fill=True, radius=2).add_to(map_osm2)

map_osm2

In [ ]:
df_staten['Economic Need Index'] = df_staten['Economic Need Index'].fillna((df_staten['Economic Need Index'].mean()))


#colors = ['red', 'yellow', 'dusty purple', 'blue', 'white', 'brown', 'green', 'purple', 'orange', 'grey', 'coral']
colors = ['chartreuse', 'limegreen', 'yellowgreen', 'y', 'olive', 'indianred', 'firebrick', 'tomamto', 'orangered', 'red']
d = (df_staten['Economic Need Index']*10).astype('int')
cols = [colors[int(i)] for i in d]


map_osm2 = folium.Map([df_staten['Latitude'].iloc[0], df_staten['Longitude'].iloc[0]], zoom_start=10.2,tiles='cartodbdark_matter')

for lat, long, col in zip(df_staten['Latitude'], df_staten['Longitude'], cols):
    #rown = list(rown)
    folium.CircleMarker([lat, long], color=col, fill=True, radius=2).add_to(map_osm2)

map_osm2

In [ ]:
df_manha.plot(kind="scatter", x="Economic Need Index", y="Percent Black", figsize=(15,9) )
plt.show()
df_manha.plot(kind="scatter", x="Economic Need Index", y="Percent Hispanic", figsize=(15,9) )
plt.show()
df_manha.plot(kind="scatter", x="Economic Need Index", y="Percent Asian", figsize=(15,9) )
plt.show()
df_manha.plot(kind="scatter", x="Economic Need Index", y="Percent White", figsize=(15,9) )
plt.show()

plt.figure(figsize=(15,10))
#ax = sns.violinplot(x="District", y="Economic Need Index", hue="Community School?", data=df_manha, palette="muted")
sns.swarmplot(x="District", y="Percent Black", hue="Community School?", data=df_manha)
plt.show()

In [ ]:
print(df_manha.shape)

ax = sns.countplot(df_manha['Supportive Environment Rating'],label="Count")       # M = 212, B = 357
plt.show()
ax2 = sns.countplot(df_manha['Community School?'],label="Count")       # M = 212, B = 357
plt.show()

print(df_manha['Grades'].unique())

Another thing worth exploring would be checking whether the amount of grades a school has is related with the amount of children that take the exam. To do so, we have to merge our two dataframes.



In [ ]:
print(registrations['School name'].unique())
print(df['School Name'].unique())

In [ ]:
df.plot(kind="scatter", x="Longitude", y="Latitude", c="Economic Need Index", cmap=plt.get_cmap("jet"),label='Schools', title='New York School Population Map',
    colorbar=True, alpha=0.4, figsize=(15,9))
plt.legend()
plt.show()

df_manha.plot(kind="scatter", x="Longitude", y="Latitude", c="Economic Need Index", cmap=plt.get_cmap("jet"),label='Schools', title='New York School Population Map',
    colorbar=True, alpha=0.4, figsize=(15,9) )
plt.legend()
plt.show()

plt.figure(figsize=(15,10))
#ax = sns.violinplot(x="District", y="Economic Need Index", hue="Community School?", data=df_manha, palette="muted")
sns.swarmplot(x="District", y="Economic Need Index", hue="Community School?", data=df_manha)
plt.show()
